In [1]:
import sys
import os

# Set project root (assumes this notebook is in ./notebooks)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)
from src.core import core

dlls = core.generate_dlls(use_contract = True)
dll = dlls['V18']
contract = dlls['Siemens.Engineering.Contract']

import clr  # noqa: E402
from System.IO import DirectoryInfo, FileInfo  # noqa: E402

clr.AddReference(dll.as_posix())
clr.AddReference(contract.as_posix())

import Siemens.Engineering as SE  # noqa: E402
import src.modules.Portals as Portals


imports = Portals.Imports(SE, DirectoryInfo, FileInfo)




In [2]:
from pathlib import Path, PurePosixPath
import json

from src.schemas import configuration

BASE_DIR = Path(".").absolute().parent
smc = BASE_DIR / "tests" / "configs" / "smc.json"

CONFIG = None
with open(smc) as file:
    CONFIG = configuration.validate(json.load(file))


In [3]:
PID = 12848

In [4]:
settings = {"enable_ui": True, "connection_method": {'mode': 'attach', "process_id": PID}}
TIA = Portals.connect(imports, CONFIG, settings)


[2025-08-06 10:17:21] INFO [src.modules.Portals:36] - Attached TIA Portal Openness (12848) WithUserInterface at 8/6/2025 2:17:21 AM


In [6]:
import src.modules.Devices as Devices
device = TIA.Projects[0].Devices[0]
plcsoftware = Devices.get_plc_software(imports, device)
plcsoftware

[2025-08-06 10:17:35] DEBUG [src.modules.Devices:44] - Accessing a PlcSoftware from Device Item Rack_0
[2025-08-06 10:17:35] DEBUG [src.modules.Devices:51] - No Software Container for Device Item Rack_0
[2025-08-06 10:17:35] DEBUG [src.modules.Devices:44] - Accessing a PlcSoftware from Device Item PLC_1
[2025-08-06 10:17:35] DEBUG [src.modules.Devices:61] - Found PlcSoftware for Device Item PLC_1


In [7]:
import src.modules.PlcBlocks as PlcBlocks


OB = PlcBlocks.find(plcsoftware, PurePosixPath("/"), "Main")